In [40]:
import matplotlib
%matplotlib inline
matplotlib.rcParams['font.size'] = 28.0
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2
import loadMWSnap as mws
import velocitySpectroscopyHelpers as vsh
reload(mws);
reload(vsh);

### Micro-X parameters ###
fwhm_MX = 3.0/(3.5*10**3) # fractional energy resolution at 3.5 keV
theta_MX = 20.0
mx_params = {
    'aperture': 1.0, # effective aperture in cm^2
    'exposure': 300.0, # exposure time in sec
    'fov': 2*np.pi*(1.-np.cos(theta_MX*np.pi/180.)), # solid angle of the FOV in str
    'sigma_energy': fwhm_MX/(2.*(2.*np.log(2.))**0.5), # Guassian sigma of the energy response
    'theta_sample': theta_MX, # angular radius of the sampling cone in degrees
}
# print mx_params
mx_lon = np.array([-105., -85., -65., -45., -25., 0., 25., 45., 65., 85., 105.])
fill_lon = np.linspace(-120.0, 120.0, 50)

In [57]:
# process all haloes using analytic vs n-body
ndata = np.array([len(mx_lon)], dtype=np.int32)
nfill = np.array([len(fill_lon)], dtype=np.int32)
haloes = mws.allHaloes[30:]
for i, h in enumerate(haloes):
    
    print 'Halo %d of %d (%d):' % (i+1, len(haloes), h)

    # load haloes
    pos, vel, partmass, ldim, data = mws.loadMWSnap(halo=h, verbose=False)
    pos -= vsh.pos_sun
    vel -= vsh.vel_sun

    print 'N-body sampling',
    mx_nbody = vsh.spectroscopy(pos, vel, partmass, lon=mx_lon, **mx_params)

    print 'Analytic sampling',
    c = data['rvir']/data['rs']
    rho0=data['mvir']/(4*np.pi*data['rs']**3*(np.log(1+c)-c/(1+c))) # solve for rho0 for NFW
    mx_analy = vsh.spectroscopy_analytic(rho0=rho0, Rs=data['rs'], Rvir=data['rvir'],\
                                         lon=mx_lon, **mx_params)
    
    mx_fill = vsh.spectroscopy_analytic(rho0=rho0, Rs=data['rs'], Rvir=data['rvir'],\
                                         lon=fill_lon, **mx_params)
    
    with open('../data/halo%d.nbody'%h, 'wb') as f:
        ndata.tofile(f)
        mx_nbody['l'].tofile(f)
        mx_nbody['cen'].tofile(f)
        mx_nbody['sig'].tofile(f)
        mx_nbody['Ns'].tofile(f)
        mx_nbody['Nb'].tofile(f)
        f.close()
        
    with open('../data/halo%d.analytic'%h, 'wb') as f:
        ndata.tofile(f)
        mx_analy['l'].tofile(f)
        mx_analy['cen'].tofile(f)
        mx_analy['sig'].tofile(f)
        mx_analy['Ns'].tofile(f)
        mx_analy['Nb'].tofile(f)
        f.close()
        
    with open('../data/halo%d.fill'%h, 'wb') as f:
        nfill.tofile(f)
        mx_fill['l'].tofile(f)
        mx_fill['cen'].tofile(f)
        mx_fill['sig'].tofile(f)
        mx_fill['Ns'].tofile(f)
        mx_fill['Nb'].tofile(f)
        f.close()

